<a href="https://colab.research.google.com/github/pachterlab/LSCHWCP_2023/blob/main/Notebooks/align_macaque_PBMC_data/1_virus_no_mask/1_align_no_host_mask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Align sequencing reads to PalmDB with kallisto translated search without masking host sequences

In [ ]:
# Install kallisto from source
!git clone -q https://github.com/pachterlab/kallisto.git
!cd kallisto && mkdir build && cd build && cmake .. && make

# Install bustools from source
!git clone -q https://github.com/BUStools/bustools.git
!cd bustools && mkdir build && cd build && cmake .. && make

# Define paths to kallisto and bustools binaries
kallisto = "/content/kallisto/build/src/kallisto"
bustools = "/content/bustools/build/src/bustools"

CMake Deprecation Warning at CMakeLists.txt:1 (cmake_minimum_required):
  Compatibility with CMake < 3.5 will be removed from a future version of
  CMake.

  Update the VERSION argument <min> value or use a ...<max> suffix to tell
  CMake that the project does not need compatibility with older versions.


-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test COMPILER_SUPPORTS_CXX17
-- Performing Test COMPILER_SUPPORTS_CXX17 - Success
shared build
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_P

In [ ]:
# Download the customized transcripts to gene mapping
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
# Download the RdRP amino acid sequences
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa

virus_fasta = "palmdb_rdrp_seqs.fa"
virus_t2g = "palmdb_clustered_t2g.txt"

--2023-12-10 01:24:34--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_clustered_t2g.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4561689 (4.3M) [text/plain]
Saving to: ‘palmdb_clustered_t2g.txt’

palmdb_clustered_t2 100%[===================>]   4.35M  --.-KB/s    in 0.1s    

2023-12-10 01:24:34 (43.0 MB/s) - ‘palmdb_clustered_t2g.txt’ saved [4561689/4561689]

--2023-12-10 01:24:34--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/PalmDB/palmdb_rdrp_seqs.fa
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaitin

In [ ]:
# Number of threads to use in alignment
threads = 2

### Download raw sequencing data

In [ ]:
!pip install -q ffq
import json

out = "GSE158390_data.json"

# # Download the complete dataset (106 paired fastqs containing a total of 30 billion reads)
# !ffq GSE158390 --ftp -o $out

# Download only two fastq pairs to demonstrate this notebook
!ffq SRR12698499 SRR12698500 --ftp -o $out

f = open(out)
data = json.load(f)
f.close()

print(len(data))

for dataset in data:
    url = dataset["url"]
    !curl -O $url

[2023-12-10 01:24:43,795]    INFO Parsing GEO GSE158390
[2023-12-10 01:24:44,397]    INFO Finding supplementary files for GEO GSE158390
[2023-12-10 01:24:46,250]    INFO No supplementary files found for GSE158390
[2023-12-10 01:24:47,633] WARNING There are 106 samples for GSE158390
[2023-12-10 01:24:47,633]    INFO Parsing GSM GSM4800305
[2023-12-10 01:24:48,035]    INFO Finding supplementary files for GSM GSM4800305
[2023-12-10 01:24:49,520]    INFO No supplementary files found for GSM4800305
[2023-12-10 01:24:50,286]    INFO Getting sample for GSM4800305
[2023-12-10 01:24:51,467]    INFO Parsing sample SRS7415341
[2023-12-10 01:24:52,102] WARNING Failed to parse sample information from ENA XML. Falling back to ENA search...
[2023-12-10 01:24:52,699]    INFO Getting Experiment for SRS7415341
[2023-12-10 01:24:52,699]    INFO Parsing Experiment SRX9178097
[2023-12-10 01:24:52,704] WARNING There is 1 run for SRX9178097
[2023-12-10 01:24:52,704]    INFO Parsing run SRR12698499
[2023-12-1

In [ ]:
import glob

fastqs = []
for filename in glob.glob("*.fastq.gz"):
    fastqs.append(filename.split("/")[-1])

fastqs.sort()

# Get sample names
samples = []
for fastq in fastqs:
    samples.append(fastq.split("_")[0])

samples = list(set(samples))

# Generate sample batch file
sample_batch_file = "batch.txt"
with open(sample_batch_file, "w") as batch_file:
    for sample in samples:
        fastq1 = sample + "_1.fastq.gz"
        fastq2 = sample + "_2.fastq.gz"
        batch_file.write(sample + "\t" + fastq1 + "\t" + fastq2 + "\n")

### Align to PalmDB

Generate virus index:

In [ ]:
virus_index = "virus_index.idx"

# Generate virus reference index
!$kallisto index \
    --aa \
    -t $threads \
    -i $virus_index \
    $virus_fasta


[build] loading fasta file palmdb_rdrp_seqs.fa
[build] k-mer length: 31
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Start computing k-mer cardinality estimations (1/2)
KmerStream::KmerStream(): Finished
CompactedDBG::build(): Estimated number of k-mers occurring at least once: 37641510
CompactedDBG::build(): Estimated number of minimizer occurring at least once: 7877811
CompactedDBG::filter(): Processed 87630084 k-mers in 296561 reads
CompactedDBG::filter(): Found 37508722 unique k-mers
CompactedDBG::filter(): Number of blocks in Bloom filter is 257317
CompactedDBG::construct(): Extract approximate unitigs (1/2)
CompactedDBG::construct(): Extract approximate unitigs (2/2)
CompactedDBG::construct(): Closed all input files

CompactedDBG::construct(): Splitting unitigs (1/2)

CompactedDBG::construct(): Splitting unitigs (2/2)
CompactedDBG::construct(): Before split: 2040572 unitigs
CompactedDBG::construct(): After split (1/1): 2

Align to PalmDB and correct barcodes using host cell barcode onlist:

In [ ]:
out_folder = "virus_no_mask_alignment_results"

In [ ]:
%%time
!$kallisto bus \
        -i $virus_index \
        -o $out_folder \
        --aa \
        -t $threads \
        -B $sample_batch_file \
        --batch-barcodes \
        -x 0,0,12:0,12,20:1,0,0


[bus] will try running read files supplied in batch file
[bus] Note: Strand option was not specified; setting it to --unstranded for specified technology
[index] k-mer length: 31
[index] number of targets: 296,561
[index] number of k-mers: 37,541,756
[quant] running in single-end mode
[quant] will process file 1: SRR12698500_1.fastq.gz
[quant] will process file 2: SRR12698500_2.fastq.gz
[quant] will process file 1: SRR12698499_1.fastq.gz
[quant] will process file 2: SRR12698499_2.fastq.gz
[progress] 575M reads processed (0.9% mapped)              done
[quant] processed 575,902,616 reads, 5,002,378 reads pseudoaligned

CPU times: user 7min 36s, sys: 1min 6s, total: 8min 42s
Wall time: 19h 40min 31s


In [ ]:
# Download cell barcode onlist generated during alignment to host
!wget https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/align_macaque_PBMC_data/bustools_onlist.txt

--2023-12-10 21:38:39--  https://raw.githubusercontent.com/pachterlab/LSCHWCP_2023/main/Notebooks/align_macaque_PBMC_data/bustools_onlist.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10413429 (9.9M) [text/plain]
Saving to: ‘bustools_onlist.txt’

bustools_onlist.txt 100%[===================>]   9.93M  --.-KB/s    in 0.1s    

2023-12-10 21:38:40 (76.6 MB/s) - ‘bustools_onlist.txt’ saved [10413429/10413429]



In [ ]:
%%time
!$bustools sort \
    -m 4G \
    -t $threads \
    -o $out_folder/output_sorted.bus \
    $out_folder/output.bus

!$bustools correct \
    -w bustools_onlist.txt \
    -o $out_folder/output_sorted_corrected.bus \
    $out_folder/output_sorted.bus

!$bustools sort \
    -m 4G \
    -t $threads \
    -o $out_folder/output_sorted_corrected_sorted.bus \
    $out_folder/output_sorted_corrected.bus

!$bustools count \
    --genecounts \
    -o $out_folder/bustools_count/ \
    -g $virus_t2g \
    -e $out_folder/matrix.ec \
    -t $out_folder/transcripts.txt \
    $out_folder/output_sorted_corrected_sorted.bus

partition time: 0.038167s
 all fits in buffer
Read in 5002378 BUS records
reading time 0.09309s
sorting time 1.04743s
writing time 0.24315s
Found 775516 barcodes in the on-list
Processed 1567608 BUS records
In on-list = 742875
Corrected    = 236336
Uncorrected  = 588397
partition time: 0.004806s
 all fits in buffer
Read in 979211 BUS records
reading time 0.008962s
sorting time 0.142693s
writing time 0.098763s
CPU times: user 136 ms, sys: 23.6 ms, total: 160 ms
Wall time: 15.2 s
